# FD equations for the cogrowth sequence of $\mathbb{Z}_2^m*\mathbb{Z}_d^n$, $d>2$, $m,n$ parameters

Genearting set is $S=\{x_1,...,x_m,x_{m+1},y_1,y_1^{-1}, ..., y_n,y_n^{-1}\}$. We denote $x$ as any $x_i,i\le m$, and $y$ as any $y_j$.

In [1]:
var('m n t')
d=9
dups = [m,n]

In [2]:
## do not change!


# Compute free group and gen set
F2.<x,y> = FreeGroup(2)



xs = [x,y]

G = F2/ tuple([v^ex for v,ex in zip(xs,[2,d])])

xs = G.gens()
x,y = xs

In [3]:
Grs = G.rewriting_system() # a rewriting system to simplfy elements
Grs.make_confluent() # this guarrantees that equivalent elements map to a UNIQUE expression

def recmap(f):
    # f: an single argument function
    def _recmap_f(obj):
        # obj: atomic element or an iterable collection of them
        # return the reduced form for the element via a single argument function f,
        #    of the same iterable type, with each element recursively reduced
        try:
            iter(obj)
        except:
            # not iterable, so must be one atomic element
            return f(obj)
        container = type(obj)
        return container(list(map(_recmap_f, obj)))
    return _recmap_f

rd = recmap(Grs.reduce) # reduce elements of G, or iterable collections of them
Grs

Rewriting system of Finitely presented group < x, y | x^2, y^9 >
with rules:
    x^-1    --->    x
    x^2    --->    1
    y^-5    --->    y^4
    y^5    --->    y^-4

In [4]:
Sis = [Set([x]),Set([rd(y),rd(y^-1)])]
S = set()
for A in Sis:
    S.update(A)
S= Set(S)
Sis,S

([{x}, {y, y^-1}], {y, y^-1, x})

In [5]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\varnothing'
    return latex(A)

In [6]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

def reset_var_space():
    global VAR_COUNTER
    var_space.clear()
    VAR_COUNTER=0
    add_series_var(goal_pair)
    
reset_var_space()

In [7]:
pretty_print(var_space)

{(1, {}): v0}

In [8]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    # assume elements all belong to G, and written in reduced from
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    return rd(C)

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [9]:
# Now get the cyclic factors


def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: one of the xi's, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume A is a finite sets
    return all((in_cyclic_factor(el,v) for el in A))

In [10]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
        assume g,X are rewritten in simplest form
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    else:
        for v,gen in zip(xs, Sis):
            if subset_of_cyclic_factor(X,v):
                if g!=one:
                    eqn = t*ZZ(g in gen)
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                else:
                    eqn = 1*t^0
                    for gen_ext, nrep in zip(Sis,dups):
                        for s in gen_ext:
                            sinv = rd(s^-1)
                            set_sinv = Set([sinv])
                            if add_series_var((sinv,set_sinv)) and ret_new_pairs:
                                newpairs.add((sinv,set_sinv))
                            # need to subtract one when the gen set is within the same class of factors as {v}\cup X
                            eqn += t*var_space[(sinv,set_sinv)]*(nrep-ZZ(gen_ext==gen))
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                
                break
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn

In [11]:
# Let us build the system of equations


reset_var_space()

system = []
queue = [goal_pair]

# use BFS
while queue:
    pair = queue.pop(0)
    #pretty_print(pair)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)

In [12]:
#Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\varnothing}(t)} = {F_{1,\varnothing}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = {\left(m - 1\right)} t {F_{x,\left\{x\right\}}(t)} + n t {F_{y^{-1},\left\{y^{-1}\right\}}(t)} + n t {F_{y,\left\{y\right\}}(t)} + t {F_{x,\left\{x\right\}}(t)} + 1 \\
{F_{x,\left\{x\right\}}(t)} = {F_{1,\left\{x\right\}}(t)} {F_{x,\left\{1, x\right\}}(t)} \\
{F_{y,\left\{y\right\}}(t)} = {F_{1,\left\{y\right\}}(t)} {F_{y,\left\{y, 1\right\}}(t)} \\
{F_{y^{-1},\left\{y^{-1}\right\}}(t)} = {F_{y^{-1},\left\{1, y^{-1}\right\}}(t)} {F_{1,\left\{y^{-1}\right\}}(t)} \\
{F_{1,\left\{x\right\}}(t)} = {\left({F_{1,\left\{1, x\right\}}(t)} - 1\right)} {F_{1,\left\{x\right\}}(t)} + 1 \\
{F_{x,\left\{1, x\right\}}(t)} = t \\
{F_{y,\left\{y, 1\right\}}(t)} = t {F_{y^{2},\left\{y, y^{2}\right\}}(t)} + t \\
{F_{1,\left\{y\right\}}(t)} = {\left({F_{1,\left\{y, 1\right\}}(t)} - 1\right)} {F_{1,\left\{y\right\}}(t)} + 1 \\
{F_{y^{-1},\left\{1, y^{-1}\right\}}(t)} = t {F_{y

v0 == v0*(v1 - 1) + 1

v1 == (m - 1)*t*v2 + n*t*v3 + n*t*v4 + t*v2 + 1

v2 == v5*v6

v4 == v7*v8

...

v159 == 0

v160 == m*t*v2 + (n - 1)*t*v3 + (n - 1)*t*v4 + 1

v161 == 0

v162 == 0

v163 == 0

In [13]:
# There are symmetries amongst the variables such that some variables will repersent the same series.
# Amongst each such equivalence class, we only need to keep one representation.
# To strip symmetry, we need union find.

from collections import defaultdict as DD

class UF(object):
    
    def __init__(self):
        self.V = {}
        self.parent = {}
    
    def add(self, node):
        if node not in self.V:
            self.V[node] = 1
            self.parent[node]=node
    
    def find(self,u):
        P = self.parent
        while P[u]!=u:
            P[u] = P[P[u]]
            u = P[u]
        return u
    
    def union(self,u,v):
        u,v = map(self.find, [u,v])
        u,v = sorted([u,v],key = lambda s: self.V[s])
        self.parent[u]=v
        self.V[v]+=self.V[u]
    
    def verts(self):
        return iter(self.V)
    
    def comps(self):
        ans = DD(set)
        for v in self.V:
            ans[self.find(v)].add(v)
        return ans
    


In [14]:
# initialize the structure to track symmetry
eqcl = UF()
for pair in var_space:
    eqcl.add(pair)

#eqcl.comps()

In [15]:
# since S is inverse closed, then (g,X) is essentially the same as (g^-1,X^-1)

vis = set()
for g,X in var_space:
    if (g,X) in vis:
        continue
    ginv = rd(g^-1)
    Xinv = Set([v^-1 for v in X])
    Xinv = rd(Xinv)
    pairinv = (ginv,Xinv)
    if pairinv in var_space:
        vis.add(pairinv)
        eqcl.union((g,X),pairinv)
#eqcl.comps()

In [16]:
from pprint import pprint
pprint(eqcl.comps())

defaultdict(<class 'set'>,
            {(1, {y^3, y^4, y^-4, y, y^2, y^-1, 1, y^-3, y^-2}): {(1,
                                                                   {y^3, y^4, y^-4, y, y^2, y^-1, 1, y^-3, y^-2})},
             (1, {y^3, y^4, y^-4, y, y^2, y^-1, y^-3, y^-2}): {(1,
                                                                {y^3, y^4, y^-4, y, y^2, y^-1, y^-3, y^-2})},
             (1, {y^3, y^-4, y^4, y^-2, y^-1, 1, y^-3, y^2}): {(1,
                                                                {y^3, y^-4, y^4, y^2, y, 1, y^-3, y^-2}),
                                                               (1,
                                                                {y^3, y^-4, y^4, y^-2, y^-1, 1, y^-3, y^2})},
             (1, {y^3, y^-4, y^4, y^-2, y^-1, y^-3, y^2}): {(1,
                                                             {y^3, y^-4, y^4, y, y^-2, y^-3, y^2}),
                                                            (1,
                                

In [17]:
# now make subtitutions to the system based on the symetries
reduced_system = []
rev_var_space = {j:i for i,j in var_space.items()}
for eq in system:
    v = eq.lhs()
    pv = rev_var_space[v]
    if pv!=eqcl.find(pv):
        continue
    ag = eq.rhs().args()
    ag = set(ag)-{t,m,n}
    ag = list(ag)
    agrepl = [rev_var_space[u] for u in ag]
    agrepl = [eqcl.find(u) for u in agrepl]
    agrepl = [var_space[u] for u in agrepl]
    reduced_system.append(v==(eq.rhs().subs(dict(zip(ag,agrepl)) )).expand().simplify())

In [18]:
for eq in reduced_system:
    pretty_print(eq)

v0 == v0*v1 - v0 + 1

v1 == m*t*v2 + 2*n*t*v4 + 1

v2 == v5*v6

v4 == v10*v9

v5 == v11*v5 - v5 + 1

...

v156 == t

v158 == 0

v160 == m*t*v2 + 2*n*t*v4 - 2*t*v4 + 1

v162 == 0

v163 == 0

In [19]:
# get rid of all F-varibles that only depend on t
while 1:
    terminals = {}
    for eq in reduced_system:
        if Set(eq.rhs().args()).issubset(Set([t])):
            terminals[eq.lhs()] = eq.rhs()
    if not terminals:
        break
    reduced_system = [eq.subs(terminals) for eq in reduced_system if eq.lhs() not in terminals]

In [20]:
for eq in reduced_system:
    pretty_print(eq)

v0 == v0*v1 - v0 + 1

v1 == m*t*v2 + 2*n*t*v4 + 1

v2 == t*v5

v4 == v10*v9

v5 == v11*v5 - v5 + 1

...

v129 == t*v127

v143 == m*t*v2 + 2*n*t*v4 + t*v144 - 2*t*v4 + 1

v144 == t*v151

v151 == v151*v160 - v151 + 1

v160 == m*t*v2 + 2*n*t*v4 - 2*t*v4 + 1

In [21]:
# Take the reduced implicit system of equations, with the list of variables to eliminate,
#  and dump them to a maple data file
import os

var_rem = {eq.lhs() for eq in reduced_system}
print(os.getcwd())
fdir = 'maple_data_files'
if not os.path.isdir(fdir):
    os.mkdir(fdir)
fsuffix = '-dat.maple'
fname = ''
for n,rep in zip([2,d],dups):
    fname+='Z%d_%s__'%(n,rep)
fname+='ic'
fname+=fsuffix
fname = os.path.join(fdir,fname)
if not os.path.exists(fname):
    fd = open(fname, 'w')
    fd.write('syst:='+str(reduced_system).replace('==','=')+';')
    fd.write('\n')
    fd.write('varlist:='+str((sorted(var_rem, key = lambda vv: int(str(vv)[1:]))))+';')
    fd.write('\n')
    fd.close()

/home/sage/OneDrive - sfu.ca/Research/Msc_math_sfu/Marni/cogrowth_computations
